In [3]:
import pandas as pd
import numpy as np
import pickle

## 2. Clean

In [4]:
dt_all_cleaned = pd.read_csv("../../data/Mercedes_Benz_Greener_Manufacturing/data/dt_all_transformed.csv")

In [5]:
dt_train_raw = dt_all_cleaned.loc[dt_all_cleaned["IsTrainTest"] == "train"]
dt_test_raw = dt_all_cleaned.loc[dt_all_cleaned["IsTrainTest"] == "test"]

### 2.1. Duplicated cols

#### 2.1.1 Drop dup cols in dt_all

In [6]:
# duplicated cols in dt_all
cols_dup_all_toDrop = dt_all_cleaned.T.duplicated()[dt_all_cleaned.T.duplicated() == True].index.values
dt_all_cleaned = dt_all_cleaned.drop(cols_dup_all_toDrop, axis = 1)

In [7]:
dt_all_cleaned.shape

(8418, 343)

#### 2.1.2 Rename the remaining dup cols

In [8]:
# duplicated cols in dt_train
cols_dup_train = dt_train_raw.T.duplicated(keep = False)[dt_train_raw.T.duplicated(keep = False) == True].index.values
# duplicated cols in dt_test
cols_dup_test = dt_test_raw.T.duplicated(keep = False)[dt_test_raw.T.duplicated(keep = False) == True].index.values

In [9]:
# change col names for cols_dup_train and cols_dup_test
dict_dup_train = {x: "dup_train_" + x for x in list(cols_dup_train)}
dt_all_cleaned = dt_all_cleaned.rename(columns = dict_dup_train)
dict_dup_test = {x: "dup_test_" + x for x in list(cols_dup_test[cols_dup_test != "y"])}
dt_all_cleaned = dt_all_cleaned.rename(columns = dict_dup_test)

In [10]:
dt_all_cleaned.shape

(8418, 343)

### 2.2 Duplicated rows

#### 2.2.1 Rename them

In [11]:
# cols_cat
cols_cat = dt_all_cleaned.drop("IsTrainTest", axis = 1).select_dtypes(include = ['object']).columns.values
# cols_int
cols_int = dt_all_cleaned.drop("ID", axis = 1).select_dtypes(include = ['int64']).columns

In [12]:
dt_all_cleaned.loc[:, "IsDupRow_All"] = dt_all_cleaned.drop(["ID", "y"], axis = 1).duplicated(keep = False).astype("int64")
dt_all_cleaned.loc[:, "IsDupRow_Cat"] = dt_all_cleaned.drop(["ID", "y"], axis = 1)[cols_cat].duplicated(keep = False).astype("int64")
dt_all_cleaned.loc[:, "IsDupRow_Int"] = dt_all_cleaned.drop(["ID", "y"], axis = 1)[cols_int].duplicated(keep = False).astype("int64")

In [13]:
dt_all_cleaned.shape

(8418, 346)

#### 2.2.2 Mean y of duplicated row(s)

### 2.3 Remove single values

In [ ]:
# single value cols in dt_train
cols_single_train = []
for col in dt_all_cleaned.loc[dt_all_cleaned["IsTrainTest"] == "train"].drop(["y", "IsTrainTest"], axis = 1).columns.values:
    len_unique = len(np.unique(dt_all_cleaned.loc[dt_all_cleaned["IsTrainTest"] == "train"][col].values))
    if len_unique == 1:
        cols_single_train.append(col)
# single value cols in dt_test
cols_single_test = []
for col in dt_all_cleaned.loc[dt_all_cleaned["IsTrainTest"] == "test"].drop(["y", "IsTrainTest"], axis = 1).columns.values:
    len_unique = len(np.unique(dt_all_cleaned.loc[dt_all_cleaned["IsTrainTest"] == "test"][col].values))
    if len_unique == 1:
        cols_single_test.append(col)

In [ ]:
# change col names for cols_single_train and cols_single_test
dict_single_train = {x: "single_train_" + x for x in cols_single_train}
dt_all_cleaned = dt_all_cleaned.rename(columns = dict_single_train)
dict_single_test = {x: "single_test_" + x for x in cols_single_test}
dt_all_cleaned = dt_all_cleaned.rename(columns = dict_single_test)

In [ ]:
dt_all_cleaned.shape

### 2.4 Remove complimentary cols

In [ ]:
cols_int = dt_all_cleaned.drop("ID", axis = 1).select_dtypes(include = ['int64']).columns

In [ ]:
def removeCompCols(dt, cols):
    seen = []
    col2s = []
    nrow = dt.shape[0]
    for col1 in cols_int:
        for col2 in cols_int:
            compliment = sum(dt[col1].values + dt[col2].values)
            same = np.sum(dt[col1] == dt[col2])
            if (compliment == nrow) & (same == 0):
                seen.append((col1, col2))
                if (col2, col1) not in seen:
                    col2s.append(col2)
                    print(col1, col2)
    return col2s

In [ ]:
cols_comp = removeCompCols(dt_all_cleaned, cols_int)

In [ ]:
dt_all_cleaned = dt_all_cleaned.drop(cols_comp, axis = 1)

In [ ]:
dt_all_cleaned.shape

### 2.5 Save cols_raw

In [ ]:
# cols_raw
cols_raw = dt_all_cleaned.drop(["ID", "y", "IsTrainTest", "IsDupRow_All", "IsDupRow_Cat", "IsDupRow_Int"
                            , "X0", "X1", "X2", "X3", "X4", "X5", "X6", "X8"], axis = 1).columns.values
pickle.dump(cols_raw, open( "../../data/Mercedes_Benz_Greener_Manufacturing/data/cols_raw.pkl", "wb"))

In [ ]:
# dt_all_cleaned
dt_all_cleaned.to_csv("../../data/Mercedes_Benz_Greener_Manufacturing/data/dt_all_cleaned.csv", index = False)

In [ ]:
dt_all_cleaned.dup